In [1]:
import cv2
# impimgBackground = cv2.imread('Frontend/background2.png')ort ultralytics
import numpy as np
import pandas as pd
import face_recognition
import os
import pickle

# Load the YOLO model
# model = ultralytics.YOLO("yolov8n-face.pt")



In [2]:
imgBackground = cv2.imread('Frontend/background2.png')

In [3]:
from pymongo import MongoClient
from datetime import datetime

def displayMultipleText(studentInfoList):

#For Textual Data
    for i, studentInfo in enumerate(studentInfoList):
            base_y = 186 + i * 117

            # For Textual Data
            cv2.putText(imgBackground, str(studentInfo['ID'].iloc[0]), (902, base_y),
                        cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)

            cv2.putText(imgBackground, str(studentInfo['Total_Attendance'].iloc[0]), (1172, base_y),
                        cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)

            cv2.putText(imgBackground, str(studentInfo['Name'].iloc[0]), (930, base_y + 38),
                        cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)

            cv2.putText(imgBackground, str(studentInfo['Field'].iloc[0]), (1127, base_y + 37),
                        cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)

    

def get_student_from_Database(student_id):
    # Connect to MongoDB
    client = MongoClient('mongodb://localhost:27017')

    # Specify the database and collection
    db = client['officialDB']
    collection = db['offficialDB']

    # Retrieve student information using find_one
    student_dataBase = collection.find_one({"ID": student_id})


    # Close the MongoDB connection
    client.close()

    return student_dataBase

def getStudentInfo(student_id, student_dataBase):
    if student_dataBase:
        # Convert the retrieved data to a DataFrame
        studentInfo = pd.DataFrame({
            "Total_Attendance": [student_dataBase["attendance"]],
            "ID": [student_dataBase["ID"]],
            "Name": [student_dataBase["Name"]],
            "Field": [student_dataBase["Dept"]],
            "Link": [student_dataBase["URL"]],
            "Last_Attendance": [student_dataBase["last_attendance"]],
        })
        
    else:
        print(f"Student with ID {student_id} not found.")
        studentInfo = pd.DataFrame({})
    # print(studentInfo)
    return studentInfo

# with update_lock:

def update_database(student_id,instance_name):
# Connect to MongoDB
    try:
        client = MongoClient('mongodb://localhost:27017')

        # Specify the database and collection
        db = client['officialDB']
        collection = db['offficialDB']

        student_to_be_updated = collection.find_one({"ID": student_id})
        # print(student_to_be_updated)
        if student_to_be_updated:

            # with update_lock:
            new_attendance_count = student_to_be_updated['attendance']+1
            new_last_attendance = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")

            collection.update_one(
                {"_id": student_to_be_updated["_id"]},
                  {
                      "$set": {
                          "Camera_Instance": instance_name,
                          "attendance": new_attendance_count,
                          "last_attendance" : new_last_attendance
                        }
                    }
            )
            updated_document = collection.find_one({"_id": student_to_be_updated["_id"]})
            # print(updated_document)
        else:
            print(f"No student found with ID: {student_id}")

    except Exception as e:
        print(f"Error updating database: {e}")

    finally:
        # Ensure that the MongoDB client is properly closed
        client.close()

def get_face_embeddings(folder_path):
    face_encodings_dict = {}

    # Iterate over the files in the folder
    for filename in os.listdir(folder_path):
        # Construct the full file path
        file_path = os.path.join(folder_path, filename)
        
        # Load the image
        image = face_recognition.load_image_file(file_path)

        # Find the face locations in the image
        face_locations = face_recognition.face_locations(image)

        # If a face is found, create the face encoding
        if len(face_locations) > 0:
            # Take the first face found
            face_location = face_locations[0]

            # Extract the face encoding
            face_encoding = face_recognition.face_encodings(image, [face_location])[0]

            # Store the face encoding in the dictionary with the filename as the key
            id=filename.split('.')[0]
            face_encodings_dict[id] = face_encoding
    return face_encodings_dict

def get_id(face_encodings_dict, face_encoding_to_check):
    min_distance = float('inf')
    min_threshold = 0.75
    name_of_person = "Unknown"   
    # Iterate over the face encodings in the dictionary
    for id, face_encoding in face_encodings_dict.items():
        # Compare the face encoding to the face encoding to check
        distance = np.linalg.norm(face_encoding - face_encoding_to_check)

                    # Check if the distance is smaller than the minimum distance
        if distance < min_distance:
            min_distance = distance
            closest_match = id
    if closest_match is not None and distance < min_threshold:
        name_of_person = closest_match
    return name_of_person

In [4]:
from collections import OrderedDict
class LRUCache:
 
    # initialising capacity
    def __init__(self, capacity: int):
        self.cache = OrderedDict()
        self.capacity = capacity
 
    # we return the value of the key
    # that is queried in O(1) and return -1 if we
    # don't find the key in out dict / cache.
    # And also move the key to the end
    # to show that it was recently used.
    def get(self, key: int) -> int:
        if key not in self.cache:
            return -1
        else:
            self.cache.move_to_end(key)
            return self.cache[key]
 
    # first, we add / update the key by conventional methods.
    # And also move the key to the end to show that it was recently used.
    # But here we will also check whether the length of our
    # ordered dictionary has exceeded our capacity,
    # If so we remove the first key (least recently used)

    def put(self, key: int, value: int) -> None:
        self.cache[key] = value
        self.cache.move_to_end(key)
        if len(self.cache) > self.capacity:
            self.cache.popitem(last = False)


In [5]:
# CREATE IMAGE folder and place all images in it for embedding
folder_path = r"D:\waleed FR\images"
students_id_and_embdding=get_face_embeddings(folder_path)

import pickle
# students_id_and_embdding=pickle.load(open("./students_id_and_embdding_linear.p", "rb"))
students_id_and_embdding=students_id_and_embdding
student_ids=[]
totalStudents = len(students_id_and_embdding)
student_id_counter = 0
print(totalStudents)

24


In [6]:
import time

class LRUCache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.cache = {}
        self.timestamps = {}

    def get(self, key):
        if key in self.cache:
            # Check if the item has expired
            if time.time() > self.timestamps[key]:
                del self.cache[key]
                del self.timestamps[key]
                return -1
            return self.cache[key]
        return -1

    def put(self, key, value,pauseForAlreadyMarked):
        if len(self.cache) >= self.capacity:
            # Find the least recently used item and remove it
            oldest_key = min(self.timestamps, key=self.timestamps.get)
            del self.cache[oldest_key]
            del self.timestamps[oldest_key]
        self.cache[key] = value
        # Store the current timestamp plus 30 seconds
        self.timestamps[key] = time.time() + pauseForAlreadyMarked


In [7]:
import cv2
import ultralytics
import numpy as np
import pandas as pd
import time
import os
from datetime import datetime
import time

# http://192.168.137.205:4747/video
cap = cv2.VideoCapture(0)

cache = LRUCache(4) 
# model = ultralytics.YOLO("yolov8n-face.pt")

imgBackground = cv2.imread('Frontend/background2.png')

folderModePath = "Frontend/Modes/"
modePathList = os.listdir(folderModePath)
imgModeList = []


for path in modePathList:
    imgModeList.append(cv2.imread(os.path.join(folderModePath,path)))

modeType = 0
totalModes = len(imgModeList)

pauseForAlreadyMarked =30
studentInfoList = []

while True:
    _, img = cap.read()

    imgBackground[118:118+480,80:80+640] = img
    imgBackground[42:42+633, 826:826+414]= imgModeList[modeType]
    
    # start = time.time()
    face_locations = face_recognition.face_locations(img)
    # print("Number of Faces Detected",len(face_locations))
    
    if len(face_locations)>0:

        # Insert Face Recognition here
        for i in range(len(face_locations)):
                # Extract the coordinates of the face location
            # top, right, bottom, left = face_location
            face_encoding = face_recognition.face_encodings(img)[i]
            temp_id= get_id(students_id_and_embdding, face_encoding)
            if temp_id != "Unknown":
                student_id = temp_id
                start1=time.time_ns()
                if modeType< totalModes and cache.get(student_id) is -1:
                    getting_time=time.time_ns()
                    print(f"{student_id} not exist in cache")
                    # print(f"time for fetching the {student_id} data(ns)",getting_time-start1)
                    modeType+=1
                    # print(f"data of {student_id} in cache",cache.get(student_id)) 
                    # if cache.get(student_id) is -1:
                    start_time=time.time()
                    student_dataBase = get_student_from_Database(student_id)
                    studentInfo = getStudentInfo(student_id, student_dataBase)
                    # print(f"Time taken to get student info {student_id}",time.time()-start_time)
                    # print("Student Info",studentInfo)
                    studentInfoList.append(studentInfo)
                    # print("length",len(studentInfoList))
                    # print("List of current students",studentInfoList)
                    start_time2=time.time_ns()
                    cache.put(student_id, studentInfo,pauseForAlreadyMarked)
                    putting_time=time.time_ns()
                    # print(f"time for putting {student_id}in  the cache (ns)",start_time2-putting_time)
                        # print("updated cache",cache.cache)
                    # else:
                    #     # print(f"value for {student_id}",cache.get(student_id))
                    #     pass
                        # print("Retaining old cache :",cache.cache)
                # else:
                #     print(f"{student_id}  exist in cache")
                #     print("student_info in cache",cache.get(student_id  ))
                #     start1=time.time_ns()
                #     print(f"time for fetching the {student_id} data(ns) ",time.time_ns()-start1)
                if modeType >= totalModes:
                    modeType=1
                    print("length",len(studentInfoList))
                    lastStudent = studentInfoList[len(studentInfoList)-1]
                    studentInfoList = []
                    studentInfoList.append(lastStudent)
                    
                    print("after last student",studentInfoList)

                #Check lastAttendance
                dateTimeObject = datetime.strptime(str(studentInfo["Last_Attendance"].iloc[0]), "%Y-%m-%d %H:%M:%S.%f")
                secondsElapsed = (datetime.now()-dateTimeObject).total_seconds()

                if secondsElapsed > pauseForAlreadyMarked:
                    update_database(student_id,"Instance1")

                imgBackground[42:42+633, 826:826+414]= imgModeList[modeType]
                
                displayMultipleText(studentInfoList)
                time.sleep(1)
        


    displayMultipleText(studentInfoList)
    # # Display the frame with the bounding boxes
    cv2.imshow('Face Attendance',imgBackground)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break


cap.release()

# Close all windows
cv2.destroyAllWindows()


<>:53: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:53: SyntaxWarning: "is" with a literal. Did you mean "=="?
c:\Users\hp\miniconda3\envs\waleed\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\hp\AppData\Local\Temp\ipykernel_26768\583892523.py:53: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if modeType< totalModes and cache.get(student_id) is -1:


K200259 not exist in cache
